# POSTGRES VIA Python

https://www.dataquest.io/blog/loading-data-into-postgres/

### Initialize environment

In [1]:
pip install psycopg2

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\mcsig_thesis\datatransfer\scripts\python.exe -m pip install --upgrade pip' command.


### Connect to postgres

In [1]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()
print("Connection open")

#close the connection
conn.close()
print("Connection closed")

Connection open
Connection closed


### Create table

In [1]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Drop table if already exists
cur.execute("DROP TABLE mensagem_stories")
print ("mensagem_stories dropped")

#Create table mensagem_stories
cur.execute("""
    CREATE TABLE mensagem_stories(
    title text,
    summary text,
    contents text,
    pub_date date,
    spatial text,
    t_begin date,
    t_end date,
    t_type text,
    temporal text,
    link text,
    section text,
    tags text,
    author text,
    publication text
)
""")
conn.commit()
print("mensagem_stories committed")

#close the connection
conn.close()
print("connection closed")

mensagem_stories dropped
mensagem_stories committed
connection closed


## Load data from CSV
Currently setting null date values to 01/01/1900

In [2]:
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()
print("Connection open")

#Load table
with open('story2.csv', 'r', encoding='utf-8') as f:
    next(f) #skip header row
    cur.copy_from(f, 'mensagem_stories', sep='|')
conn.commit()
print("data loaded")

cur.execute("""
    ALTER TABLE mensagem_stories ADD id serial PRIMARY KEY, ADD load_id text DEFAULT 'A'
""")

cur.execute("""
    UPDATE mensagem_stories
    set load_id = CONCAT('A',id)
""")
conn.commit()
print("load id assigned")

cur.execute('SELECT title,author,load_id,id FROM mensagem_stories')
all = cur.fetchall()
print(all)

conn.close()
print("connection closed")

Connection open
data loaded
load id assigned
[('Esta cidade ja é para animais vadios: gatos de rua têm abrigos para dormir', 'Maria João Martins', 'A1', 1), ('A avó da Uber Eats em Alfama é a salvaçao dos mais velhos', 'Álvaro Filho', 'A2', 2), ('Alvalade: o bairro que não é para turistas tem resistido melhor à crise da pendemia', 'Frederico Raposo ', 'A3', 3), ('"As ""Gaulesas"" que defendem a Mouraria da especulação imobiliária"', 'Catarina Reis', 'A4', 4), ('Mais floreiras e menos carros. Estes lisboetas mudaram uma rua com um canteiro.', 'Frederico Raposo ', 'A5', 5), ('Nos Anjos, o ativismo passou a fazer-se ao microfone da rádio Gabriela', 'Álvaro Filho', 'A6', 6), ('Quem é Alexis Lapas, o peixeiro de Benfica que dá polémica no Twitter', 'Álvaro Filho', 'A7', 7), ('A guerra das bicicletas passou para a Avenida de Berna e está a politizar-se', 'Frederico Raposo ', 'A8', 8)]
connection closed


## Create Relational Database model

In [3]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS story CASCADE")
print ("story dropped")

#Create table story
cur.execute("""
    CREATE TABLE story(
    title text NOT NULL,
    summary text,
    contents text,
    web_link text,
    publish_date date,
    load_id text,
    story_id serial PRIMARY KEY
)
""")
conn.commit()
print("story created")


#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS author CASCADE")
print ("author dropped")

#Create table author
cur.execute("""
    CREATE TABLE author(
    author_name text NOT NULL,
    author_id serial PRIMARY KEY
)
""")
conn.commit()
print("author created")


#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS authorship")
print ("authorship dropped")

#Create table mensagem_stories
cur.execute("""
    CREATE TABLE authorship(
    story_id INT NOT NULL,
    author_id INT NOT NULL,
    PRIMARY KEY (story_id, author_id),
    FOREIGN KEY (story_id) REFERENCES story(story_id) ON UPDATE CASCADE,
    FOREIGN KEY (author_id) REFERENCES author(author_id) ON UPDATE CASCADE
)
""")
conn.commit()
print("authorship created")

#close the connection
conn.close()
print("connection closed")

story dropped
story created
author dropped
author created
authorship dropped
authorship created
connection closed


## Distribute into database

In [4]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Distribute mensagem stories into stories
cur.execute("""
    INSERT INTO story(title, summary, contents, web_link, publish_date, load_id)
    SELECT title, summary, contents, link, pub_date, load_id
    FROM mensagem_stories
""")
conn.commit()
print("stories loaded")

cur.execute('SELECT title,load_id,story_id FROM story')
all = cur.fetchall()
print(all)

#Distribute mensagem authors into authors
cur.execute("""
    INSERT INTO author(author_name)
    SELECT DISTINCT author
    FROM mensagem_stories
""")
conn.commit()
print("authors loaded")

cur.execute('SELECT * FROM author')
all = cur.fetchall()
print(all)

#close the connection
conn.close()
print("connection closed")

stories loaded
[('Esta cidade ja é para animais vadios: gatos de rua têm abrigos para dormir', 'A1', 1), ('A avó da Uber Eats em Alfama é a salvaçao dos mais velhos', 'A2', 2), ('Alvalade: o bairro que não é para turistas tem resistido melhor à crise da pendemia', 'A3', 3), ('"As ""Gaulesas"" que defendem a Mouraria da especulação imobiliária"', 'A4', 4), ('Mais floreiras e menos carros. Estes lisboetas mudaram uma rua com um canteiro.', 'A5', 5), ('Nos Anjos, o ativismo passou a fazer-se ao microfone da rádio Gabriela', 'A6', 6), ('Quem é Alexis Lapas, o peixeiro de Benfica que dá polémica no Twitter', 'A7', 7), ('A guerra das bicicletas passou para a Avenida de Berna e está a politizar-se', 'A8', 8)]
authors loaded
[('Álvaro Filho', 1), ('Frederico Raposo ', 2), ('Catarina Reis', 3), ('Maria João Martins', 4)]
connection closed


## Populate authorship

In [5]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()
print("Connection open")

#Relate stories and authors
cur.execute("""
    INSERT INTO authorship(story_id,author_id)
    SELECT mensagem_stories.id, author.author_id
    FROM mensagem_stories
    LEFT JOIN author ON mensagem_stories.author = author.author_name
    LEFT JOIN story ON mensagem_stories.load_id = story.load_id
    ORDER BY mensagem_stories.id
""")
conn.commit()
cur.execute('SELECT * FROM authorship')
all = cur.fetchall()
print(all)


#close the connection
conn.close()
print("Connection closed")

Connection open
[(1, 4), (2, 1), (3, 2), (4, 3), (5, 2), (6, 1), (7, 1), (8, 2)]
Connection closed


## See all articles by author

In [6]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()
print("Connection open")

cur.execute("""
    SELECT author.author_name, story.title
    FROM author
    LEFT JOIN authorship ON author.author_id = authorship.author_id
    INNER JOIN story ON authorship.story_id = story.story_id
    ORDER BY author.author_name
""")
all = cur.fetchall()
print(all)


#close the connection
conn.close()
print("Connection closed")

Connection open
[('Álvaro Filho', 'A avó da Uber Eats em Alfama é a salvaçao dos mais velhos'), ('Álvaro Filho', 'Nos Anjos, o ativismo passou a fazer-se ao microfone da rádio Gabriela'), ('Álvaro Filho', 'Quem é Alexis Lapas, o peixeiro de Benfica que dá polémica no Twitter'), ('Catarina Reis', '"As ""Gaulesas"" que defendem a Mouraria da especulação imobiliária"'), ('Frederico Raposo ', 'A guerra das bicicletas passou para a Avenida de Berna e está a politizar-se'), ('Frederico Raposo ', 'Alvalade: o bairro que não é para turistas tem resistido melhor à crise da pendemia'), ('Frederico Raposo ', 'Mais floreiras e menos carros. Estes lisboetas mudaram uma rua com um canteiro.'), ('Maria João Martins', 'Esta cidade ja é para animais vadios: gatos de rua têm abrigos para dormir')]
Connection closed


## Add Publisher / Publishing

In [7]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS publisher CASCADE")
print ("publisher dropped")


#Create table publisher
cur.execute("""
    CREATE TABLE publisher(
    publisher_name text NOT NULL,
    publisher_site text,
    publisher_description text,
    publisher_id serial PRIMARY KEY
)
""")
conn.commit()
print("publisher created")


#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS publishing")
print ("publishing dropped")

#Create table publishing
cur.execute("""
    CREATE TABLE publishing(
    story_id INT NOT NULL,
    publisher_id INT NOT NULL,
    PRIMARY KEY (story_id, publisher_id),
    FOREIGN KEY (story_id) REFERENCES story(story_id) ON UPDATE CASCADE,
    FOREIGN KEY (publisher_id) REFERENCES publisher(publisher_id) ON UPDATE CASCADE
)
""")
conn.commit()
print("publishing created")

#Distribute publishers
cur.execute("""
    INSERT INTO publisher(publisher_name)
    SELECT DISTINCT publication
    FROM mensagem_stories
""")
conn.commit()
print("publishers loaded")

cur.execute('SELECT * FROM publisher')
all = cur.fetchall()
print(all)

#Relate stories and authors
cur.execute("""
    INSERT INTO publishing(story_id,publisher_id)
    SELECT mensagem_stories.id, publisher.publisher_id
    FROM mensagem_stories
    LEFT JOIN publisher ON mensagem_stories.publication = publisher.publisher_name
    LEFT JOIN story ON mensagem_stories.load_id = story.load_id
    ORDER BY mensagem_stories.id
""")
conn.commit()
cur.execute('SELECT * FROM publishing')
all = cur.fetchall()
print(all)

cur.execute("""
    SELECT publisher.publisher_name, story.title
    FROM publisher
    LEFT JOIN publishing ON publisher.publisher_id = publishing.publisher_id
    INNER JOIN story ON publishing.story_id = story.story_id
    ORDER BY publisher.publisher_name
""")
all = cur.fetchall()
print(all)


#close the connection
conn.close()
print("connection closed")

publisher dropped
publisher created
publishing dropped
publishing created
publishers loaded
[('mensagem', None, None, 1)]
[(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]
[('mensagem', 'Esta cidade ja é para animais vadios: gatos de rua têm abrigos para dormir'), ('mensagem', 'A avó da Uber Eats em Alfama é a salvaçao dos mais velhos'), ('mensagem', 'Alvalade: o bairro que não é para turistas tem resistido melhor à crise da pendemia'), ('mensagem', '"As ""Gaulesas"" que defendem a Mouraria da especulação imobiliária"'), ('mensagem', 'Mais floreiras e menos carros. Estes lisboetas mudaram uma rua com um canteiro.'), ('mensagem', 'Nos Anjos, o ativismo passou a fazer-se ao microfone da rádio Gabriela'), ('mensagem', 'Quem é Alexis Lapas, o peixeiro de Benfica que dá polémica no Twitter'), ('mensagem', 'A guerra das bicicletas passou para a Avenida de Berna e está a politizar-se')]
connection closed


## Add Section / Sectioning

In [8]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS section CASCADE")
print ("section dropped")


#Create table publisher
cur.execute("""
    CREATE TABLE section(
    section_name text NOT NULL,
    section_id serial PRIMARY KEY
)
""")
conn.commit()
print("section created")


#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS sectioning")
print ("sectioning dropped")

#Create table publishing
cur.execute("""
    CREATE TABLE sectioning(
    story_id INT NOT NULL,
    section_id INT NOT NULL,
    PRIMARY KEY (story_id, section_id),
    FOREIGN KEY (story_id) REFERENCES story(story_id) ON UPDATE CASCADE,
    FOREIGN KEY (section_id) REFERENCES section(section_id) ON UPDATE CASCADE
)
""")
conn.commit()
print("sectioning created")

#Distribute publishers
cur.execute("""
    INSERT INTO section(section_name)
    SELECT DISTINCT section
    FROM mensagem_stories
""")
conn.commit()
print("sections loaded")

cur.execute('SELECT * FROM section')
all = cur.fetchall()
print(all)

#Relate stories and sections
cur.execute("""
    INSERT INTO sectioning(story_id,section_id)
    SELECT mensagem_stories.id, section.section_id
    FROM mensagem_stories
    LEFT JOIN section ON mensagem_stories.section = section.section_name
    LEFT JOIN story ON mensagem_stories.load_id = story.load_id
    ORDER BY mensagem_stories.id
""")
conn.commit()
cur.execute('SELECT * FROM sectioning')
all = cur.fetchall()
print(all)

cur.execute("""
    SELECT section.section_name, story.title
    FROM section
    LEFT JOIN sectioning ON section.section_id = sectioning.section_id
    INNER JOIN story ON sectioning.story_id = story.story_id
    ORDER BY section.section_name
""")
all = cur.fetchall()
print(all)


#close the connection
conn.close()
print("connection closed")

section dropped
section created
sectioning dropped
sectioning created
sections loaded
[('cidade', 1), ('bairros', 2)]
[(1, 2), (2, 2), (3, 2), (4, 2), (5, 2), (6, 2), (7, 2), (8, 1)]
[('bairros', 'Esta cidade ja é para animais vadios: gatos de rua têm abrigos para dormir'), ('bairros', 'A avó da Uber Eats em Alfama é a salvaçao dos mais velhos'), ('bairros', 'Alvalade: o bairro que não é para turistas tem resistido melhor à crise da pendemia'), ('bairros', '"As ""Gaulesas"" que defendem a Mouraria da especulação imobiliária"'), ('bairros', 'Mais floreiras e menos carros. Estes lisboetas mudaram uma rua com um canteiro.'), ('bairros', 'Nos Anjos, o ativismo passou a fazer-se ao microfone da rádio Gabriela'), ('bairros', 'Quem é Alexis Lapas, o peixeiro de Benfica que dá polémica no Twitter'), ('cidade', 'A guerra das bicicletas passou para a Avenida de Berna e está a politizar-se')]
connection closed


## Theme / Theming (Tags)

In [9]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS theme CASCADE")
print ("theme dropped")


#Create table theme
cur.execute("""
    CREATE TABLE theme(
    theme_name text NOT NULL,
    theme_id serial PRIMARY KEY
)
""")
conn.commit()
print("theme created")


#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS stheming")
cur.execute("DROP TABLE IF EXISTS theming")
print ("theming dropped")

#Create table theming
cur.execute("""
    CREATE TABLE theming(
    story_id INT NOT NULL,
    theme_id INT NOT NULL,
    PRIMARY KEY (story_id, theme_id),
    FOREIGN KEY (story_id) REFERENCES story(story_id) ON UPDATE CASCADE,
    FOREIGN KEY (theme_id) REFERENCES theme(theme_id) ON UPDATE CASCADE
)
""")
conn.commit()
print("theming created")

#Distribute publishers
cur.execute("""
    INSERT INTO theme(theme_name)
    SELECT DISTINCT tags
    FROM mensagem_stories
""")
conn.commit()
print("themes loaded")

cur.execute('SELECT * FROM theme')
all = cur.fetchall()
print(all)

#Relate stories and themes
cur.execute("""
    INSERT INTO theming(story_id,theme_id)
    SELECT mensagem_stories.id, theme.theme_id
    FROM mensagem_stories
    LEFT JOIN theme ON mensagem_stories.tags = theme.theme_name
    LEFT JOIN story ON mensagem_stories.load_id = story.load_id
    ORDER BY mensagem_stories.id
""")
conn.commit()
cur.execute('SELECT * FROM theming')
all = cur.fetchall()
print(all)

cur.execute("""
    SELECT theme.theme_name, story.title
    FROM theme
    LEFT JOIN theming ON theme.theme_id = theming.theme_id
    INNER JOIN story ON theming.story_id = story.story_id
    ORDER BY theme.theme_name
""")
all = cur.fetchall()
print(all)


#close the connection
conn.close()
print("connection closed")

theme dropped
theme created
theming dropped
theming created
themes loaded
[('', 1)]
[(1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1)]
[('', 'Esta cidade ja é para animais vadios: gatos de rua têm abrigos para dormir'), ('', 'A avó da Uber Eats em Alfama é a salvaçao dos mais velhos'), ('', 'Alvalade: o bairro que não é para turistas tem resistido melhor à crise da pendemia'), ('', '"As ""Gaulesas"" que defendem a Mouraria da especulação imobiliária"'), ('', 'Mais floreiras e menos carros. Estes lisboetas mudaram uma rua com um canteiro.'), ('', 'Nos Anjos, o ativismo passou a fazer-se ao microfone da rádio Gabriela'), ('', 'Quem é Alexis Lapas, o peixeiro de Benfica que dá polémica no Twitter'), ('', 'A guerra das bicicletas passou para a Avenida de Berna e está a politizar-se')]
connection closed


## Employment

In [10]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS employment CASCADE")
print ("employment dropped")


#Create table employment
cur.execute("""
    CREATE TABLE employment(
    publisher_id INT NOT NULL,
    author_id INT NOT NULL,
    PRIMARY KEY (publisher_id, author_id),
    FOREIGN KEY (publisher_id) REFERENCES publisher(publisher_id) ON UPDATE CASCADE,
    FOREIGN KEY (author_id) REFERENCES author(author_id) ON UPDATE CASCADE
)
""")
conn.commit()
print("employment created")


#Relate authors and publishers
cur.execute("""
    INSERT INTO employment(publisher_id,author_id)
    SELECT DISTINCT publisher.publisher_id, author.author_id
    FROM mensagem_stories
    LEFT JOIN author ON mensagem_stories.author = author.author_name
    LEFT JOIN publisher ON mensagem_stories.publication = publisher.publisher_name
    ORDER BY author.author_id
""")
conn.commit()
cur.execute('SELECT * FROM employment')
all = cur.fetchall()
print(all)

cur.execute("""
    SELECT author.author_name, publisher.publisher_name
    FROM author
    LEFT JOIN employment ON author.author_id = employment.author_id
    INNER JOIN publisher ON employment.publisher_id = publisher.publisher_id
    ORDER BY author.author_name
""")
all = cur.fetchall()
print(all)


#close the connection
conn.close()
print("connection closed")

employment dropped
employment created
[(1, 1), (1, 2), (1, 3), (1, 4)]
[('Álvaro Filho', 'mensagem'), ('Catarina Reis', 'mensagem'), ('Frederico Raposo ', 'mensagem'), ('Maria João Martins', 'mensagem')]
connection closed


## Story Relations

In [11]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS storyref CASCADE")
print ("storyref dropped")


#Create table storyref
cur.execute("""
    CREATE TABLE storyref(
    storya_id INT NOT NULL,
    storyb_id INT NOT NULL,
    PRIMARY KEY (storya_id, storyb_id),
    FOREIGN KEY (storya_id) REFERENCES story(story_id) ON UPDATE CASCADE,
    FOREIGN KEY (storyb_id) REFERENCES story(story_id) ON UPDATE CASCADE
)
""")
conn.commit()
print("storyref created")

print("storyref empty")


#close the connection
conn.close()
print("connection closed")

storyref dropped
storyref created
storyref empty
connection closed


## Story typing

In [12]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS stype CASCADE")
print ("stype dropped")


#Create table theme
cur.execute("""
    CREATE TABLE stype(
    stype_name text NOT NULL,
    stype_description text,
    stype_id serial PRIMARY KEY
)
""")
conn.commit()
print("stype created")


#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS styping")
print ("styping dropped")

#Create table theming
cur.execute("""
    CREATE TABLE styping(
    story_id INT NOT NULL,
    stype_id INT NOT NULL,
    PRIMARY KEY (story_id, stype_id),
    FOREIGN KEY (story_id) REFERENCES story(story_id) ON UPDATE CASCADE,
    FOREIGN KEY (stype_id) REFERENCES stype(stype_id) ON UPDATE CASCADE
)
""")
conn.commit()
print("styping created")

print("styping empty")

#close the connection
conn.close()
print("connection closed")

stype dropped
stype created
styping dropped
styping created
styping empty
connection closed


## Instance

In [13]:
#Connect to postgres
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS sinstance CASCADE")
print ("sinstance dropped")

#Create table sistance. REQUIRES REFERENCE TO GAZETTEERS
cur.execute("""
    CREATE TABLE sinstance(
    story_id INT NOT NULL,
    t_begin DATE,
    t_end DATE,
    t_type text,
    t_desc text,
    place_type INT,
    place_id INT,
    place_desc text,
    sistance_id serial PRIMARY KEY,
    FOREIGN KEY (story_id) REFERENCES story(story_id) ON UPDATE CASCADE
)
""")
conn.commit()
print("sinstance created")

#Distribute instances
cur.execute("""
    INSERT INTO sinstance(story_id, t_begin, t_end, t_type, t_desc, place_desc)
    SELECT id, t_begin, t_end, t_type, temporal, spatial
    FROM mensagem_stories
""")
conn.commit()
print("instances pre-loaded")

cur.execute('SELECT * FROM sinstance')
all = cur.fetchall()
print(all)


#close the connection
conn.close()
print("connection closed")

sinstance dropped
sinstance created
instances pre-loaded
[(1, datetime.date(1900, 1, 1), datetime.date(1900, 1, 1), 'na', '', None, None, 'Rua do Garrido (no bairro do Alto do Pina, freguesia do Areeiro); Freguesia de Areeiro, Freguesia da Penha de França', 1), (2, datetime.date(2020, 3, 1), datetime.date(1900, 1, 1), 'ongoing', 'COVID', None, None, 'Mercearia Castanheira, Travessa do Terreiro do Trigo, 5, Alfama, Lisboa', 2), (3, datetime.date(2020, 3, 1), datetime.date(1900, 1, 1), 'ongoing', 'COVID', None, None, 'freguesia de Alvalade; Isco padaria; ', 3), (4, datetime.date(2016, 1, 1), datetime.date(1900, 1, 1), 'ongoing', '2016-present', None, None, 'bairro de Mouraria, Rua dos Lagares', 4), (5, datetime.date(2021, 1, 19), datetime.date(2021, 1, 19), 'concrete', '19-Jan', None, None, 'Rua da Cruz da Carreira', 5), (6, datetime.date(2020, 3, 1), datetime.date(1900, 1, 1), 'ongoing', 'March 2020 - current', None, None, 'Coletivo Sirigaita, 12F da Rua dos Anjos', 6), (7, datetime.dat

## Import GeoNames into Postgres
https://gist.github.com/EspadaV8/1357237/25a81f06fd1d04b54cdda35a53f359c45aefce6a

In [1]:
import wget
import zipfile
import os
import shutil

#tidy up the folder structure
#os.remove(r"C:\mcsig_git\data_load\PT.zip")
os.remove(r"C:\mcsig_git\data_load\geonamesPT.zip")
os.remove(r"C:\mcsig_git\data_load\altPT.zip")
os.remove(r"C:\mcsig_git\data_load\zipPT.zip")
os.remove(r"C:\mcsig_git\data_load\zipAll.zip")
shutil.rmtree("C:\mcsig_git\data_load\pt")
os.mkdir("pt")
print('extraneous files removed')

#download geonamesPT
print('Beginning GEONAMES PT download with wget module')
url = 'http://download.geonames.org/export/dump/PT.zip'
wget.download(url, 'C:\mcsig_git\data_load')
import zipfile
with zipfile.ZipFile('C:\mcsig_git\data_load\PT.zip', 'r') as zip_ref:
                     zip_ref.extractall('C:\mcsig_git\data_load\pt')
print('Geonames PT unzipped')
os.rename(r'C:\mcsig_git\data_load\PT.zip',r'C:\mcsig_git\data_load\geonamesPT.zip')
os.rename(r'C:\mcsig_git\data_load\pt\PT.txt',r'C:\mcsig_git\data_load\pt\geonamesPT.txt')
print('Geonames PT txt and zip renamed')


#download alternatenamesPT
print('Beginning ALTNAMES PT download with wget module')
url = 'http://download.geonames.org/export/dump/alternatenames/PT.zip'
wget.download(url, 'C:\mcsig_git\data_load')
print('Download Alternate Names PT complete')
with zipfile.ZipFile('C:\mcsig_git\data_load\PT.zip', 'r') as zip_ref:
                     zip_ref.extractall('C:\mcsig_git\data_load\pt')
print('Altnames unzipped')
os.rename(r'C:\mcsig_git\data_load\PT.zip',r'C:\mcsig_git\data_load\altPT.zip')
os.rename(r'C:\mcsig_git\data_load\pt\PT.txt',r'C:\mcsig_git\data_load\pt\altPT.txt')
print('Altnames PT txt and zip renamed')

#download postcodesPT
print('Beginning Postcodes PT download with wget module')
url = 'http://download.geonames.org/export/zip/PT.zip'
wget.download(url, 'C:\mcsig_git\data_load')
import zipfile
with zipfile.ZipFile('C:\mcsig_git\data_load\PT.zip', 'r') as zip_ref:
                     zip_ref.extractall('C:\mcsig_git\data_load\pt')
print('postcodes PT unzipped')
os.rename(r'C:\mcsig_git\data_load\PT.zip',r'C:\mcsig_git\data_load\zipPT.zip')
os.rename(r'C:\mcsig_git\data_load\pt\PT.txt',r'C:\mcsig_git\data_load\pt\zipPT.txt')
print('Postcodes PT txt and zip renamed')

#download postcodes all
#print('Beginning Postcodes all download with wget module')
#url = 'http://download.geonames.org/export/zip/allCountries.zip'
#wget.download(url, 'C:\mcsig_git\data_load')
#import zipfile
#with zipfile.ZipFile(r'C:\mcsig_git\data_load\allCountries.zip', 'r') as zip_ref:
#                     zip_ref.extractall('C:\mcsig_git\data_load\pt')
#print('postcodes all unzipped')
#os.rename(r'C:\mcsig_git\data_load\allCountries.zip',r'C:\mcsig_git\data_load\zipAll.zip')
#os.rename(r'C:\mcsig_git\data_load\pt\allCountries.txt',r'C:\mcsig_git\data_load\pt\zipAll.txt')
#print('Postcodes ALL txt and zip renamed')

#download iso-language codes
print('Beginning ISO Language Codes download with wget module')
url = 'http://download.geonames.org/export/dump/iso-languagecodes.txt'
wget.download(url, 'C:\mcsig_git\data_load\pt')
print('Download Language Codes complete')

#download featurecode
print('Beginning Feature Codes download with wget module')
url = 'http://download.geonames.org/export/dump/featureCodes_en.txt'
wget.download(url, 'C:\mcsig_git\data_load\pt')
print('Download Feature Codes complete')

#download admin1codes
print('Beginning Admin1 Codes download with wget module')
url = 'http://download.geonames.org/export/dump/admin1CodesASCII.txt'
wget.download(url, 'C:\mcsig_git\data_load\pt')
print('Download Admin1 Codes complete')

#download admin2codes
print('Beginning Admin2 Codes download with wget module')
url = 'http://download.geonames.org/export/dump/admin2Codes.txt'
wget.download(url, 'C:\mcsig_git\data_load\pt')
print('Download Admin2 Codes complete')

print('GREAT SUCCESS!!')

extraneous files removed
Beginning GEONAMES PT download with wget module
100% [###############################################################]           1M / 1MGeonames PT unzipped
Geonames PT txt and zip renamed
Beginning ALTNAMES PT download with wget module
100% [#################################################################]     201K / 201KDownload Alternate Names PT complete
Altnames unzipped
Altnames PT txt and zip renamed
Beginning Postcodes PT download with wget module
100% [###############################################################]           1M / 1Mpostcodes PT unzipped
Postcodes PT txt and zip renamed
Beginning ISO Language Codes download with wget module
100% [#################################################################]     126K / 126KDownload Language Codes complete
Beginning Feature Codes download with wget module
100% [###################################################################]     56K / 56KDownload Feature Codes complete
Beginning Admin1 Codes do

In [1]:
import psycopg2
conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()
print("Connection open")

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS geoname CASCADE")
print ("geoname dropped")

#Create table if exists
cur.execute("""
    CREATE TABLE geoname(
    geoname_id int,
    name varchar(200),
    asciiname varchar(200),
    alternatenames varchar(10000),
    latitude float,
    longitude float,
    fclass char(1),
    fcode varchar(10),
    countrycode char(2),
    cc2 varchar(200),
    admin1 varchar(20),
    admin2 varchar(80),
    admin3 varchar(20),
    admin4 varchar(20),
    population bigint,
    elevation bigint,
    gtopo30 int,
    timezone varchar(40),
    moddate date,
    PRIMARY KEY (geoname_id)
)
""")
conn.commit()
print("geonames created")

with open(r'C:\mcsig_git\data_load\pt\geonamesPT.txt', 'r', encoding='utf-8') as f:
#    next(f) #skip header row
    cur.copy_from(f, 'geoname', sep="\t", null="")
conn.commit()
print("Geonames loaded")


#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS alternatename CASCADE")
print ("alternatename dropped")

#Create table if exists
cur.execute("""
    CREATE TABLE alternatename(
    alternatename_id int,
    geonameid int,
    isoLanguage varchar(7),
    alternateName varchar(400),
    isPreferredName boolean,
    isShortName boolean,
    isColloquial boolean,
    isHistoric boolean,
    fromPeriod text,
    toPeriod text,
    PRIMARY KEY (alternatename_id),
    FOREIGN KEY (geonameid) REFERENCES geoname(geoname_id) ON UPDATE CASCADE
)
""")
conn.commit()
print("alternatenames created")

with open(r'C:\mcsig_git\data_load\pt\altPT.txt', 'r', encoding='utf-8') as f:
#    next(f) #skip header row
    cur.copy_from(f, 'alternatename', sep="\t", null="")
conn.commit()
print("alternatename loaded")



#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS iso_languagecodes CASCADE")
print ("iso_languagecodes dropped")

#Create table if exists
cur.execute("""
    CREATE TABLE iso_languagecodes(
    iso_639_3 char(4),
    iso_639_2 VARCHAR(50),
    iso_639_1 VARCHAR(50),
    language_name VARCHAR(200)
)
""")
conn.commit()
print("iso_languagecodes created")

with open(r'C:\mcsig_git\data_load\pt\iso-languagecodes.txt', 'r', encoding='utf-8') as f:
    next(f) #skip header row
    cur.copy_from(f, 'iso_languagecodes', sep="\t", null="")
conn.commit()
print("iso_languagecodes loaded")

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS admin1CodesAscii CASCADE")
print ("admin1CodesAscii dropped")

#Create table if exists
cur.execute("""
    CREATE TABLE admin1CodesAscii(
    code CHAR(20),
    name TEXT,
    nameAscii TEXT,
    geonameid int
)
""")
conn.commit()
print("admin1CodesAscii created")

with open(r'C:\mcsig_git\data_load\pt\admin1CodesASCII.txt', 'r', encoding='utf-8') as f:
#    next(f) #skip header row
    cur.copy_from(f, 'admin1CodesAscii', sep="\t", null="")
conn.commit()
print("admin1CodesAscii loaded")

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS admin2CodesAscii CASCADE")
print ("admin2CodesAscii dropped")

#Create table if exists
cur.execute("""
    CREATE TABLE admin2CodesAscii(
    code CHAR(80),
    name TEXT,
    nameAscii TEXT,
    geonameid int
)
""")
conn.commit()
print("admin2CodesAscii created")

with open(r'C:\mcsig_git\data_load\pt\admin2Codes.txt', 'r', encoding='utf-8') as f:
#    next(f) #skip header row
    cur.copy_from(f, 'admin2CodesAscii', sep="\t", null="")
conn.commit()
print("admin2CodesAscii loaded")

#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS featureCodes CASCADE")
print ("featureCodes dropped")

#Create table if exists
cur.execute("""
    CREATE TABLE featureCodes(
    code CHAR(7),
    name varchar(200),
    description TEXT
)
""")
conn.commit()
print("featureCodes created")

with open(r'C:\mcsig_git\data_load\pt\featureCodes_en.txt', 'r', encoding='utf-8') as f:
#    next(f) #skip header row
    cur.copy_from(f, 'featureCodes', sep="\t", null="")
conn.commit()
print("featureCodes loaded")


#Drop table if already exists
cur.execute("DROP TABLE IF EXISTS postalCodes CASCADE")
print ("postalCodes dropped")

#Create table if exists
cur.execute("""
    CREATE TABLE postalCodes(
    countrycode char(2),
    postalcode varchar(20),
    placename varchar(180),
    admin1name varchar(100),
    admin1code varchar(20),
    admin2name varchar(100),
    admin2code varchar(20),
    admin3name varchar(100),
    admin3code varchar(20),
    latitude float,
    longitude float,
    accuracy smallint
)
""")
conn.commit()
print("postalCodes created")

with open(r'C:\mcsig_git\data_load\pt\zipPT.txt', 'r', encoding='utf-8') as f:
#    next(f) #skip header row
    cur.copy_from(f, 'postalCodes', sep="\t", null="")
conn.commit()
print("postalCodes loaded")

cur.execute("CREATE INDEX index_alternatename_geonameid ON alternatename USING hash (geonameid)")
conn.commit()
print("alternatename indexed on geonameid")

print("GREAT SUCCESS!!!")

#Closing the connection
conn.close()

Connection open
geoname dropped
geonames created
Geonames loaded
alternatename dropped
alternatenames created
alternatename loaded
iso_languagecodes dropped
iso_languagecodes created
iso_languagecodes loaded
admin1CodesAscii dropped
admin1CodesAscii created
admin1CodesAscii loaded
admin2CodesAscii dropped
admin2CodesAscii created
admin2CodesAscii loaded
featureCodes dropped
featureCodes created
featureCodes loaded
postalCodes dropped
postalCodes created
postalCodes loaded
alternatename indexed on geonameid
GREAT SUCCESS!!!


## Visualize distribution

In [58]:
import os
from osgeo import ogr, osr
from shapely import geometry #, wkt
import geojson
import folium
import psycopg2

#define variables
mapGaz = [] #define an empty list for coordinates of Gazetteer locations

#define functions
def placeGaz(coords):
    for i in coords:
        pointGaz = geometry.Point(i[1],i[0])
        mapGaz.append(pointGaz)
    return mapGaz
    

conn = psycopg2.connect("host=localhost dbname=postgres user=postgres password=thesis2021")
cur = conn.cursor()
print("Connection open")

cur.execute("SELECT latitude,longitude,name FROM geoname WHERE admin1 = '14' OR admin1 = '19' ") # 
#cur.execute("SELECT latitude,longitude,name FROM geoname")
coordsGaz = cur.fetchall()
#print(coordsGaz)
print("number of coordsGaz: " + str(len(coordsGaz)))
placeGaz(coordsGaz)
print("number of placeGaz: " + str(len(mapGaz)))
#print("mapGaz: " + str(mapGaz))#remove in production


multiGaz = geometry.MultiPoint(mapGaz) #Associate all gazetteer locations into a variable of type multipoint
multiJson = geojson.Feature(geometry=multiGaz, properties={})
#print("multiJson: " + str(multiJson))#remove in production


print("GREAT SUCCESS!!!")

#Closing the connection
conn.close()

Connection open
number of coordsGaz: 4449
number of placeGaz: 4449
GREAT SUCCESS!!!


## Visualize Gazetteer

In [61]:
mapDisp = folium.Map(location=[38.7223,-9.1393], tiles ='Stamen Terrain', zoom_start=8) #define map style
folium.GeoJson(multiJson).add_to(mapDisp) #map all gazetteer points
#mapDisp #heavy to visualize

## Polygon gazetteer
In Dev

In [80]:
import wget
import os
import zipfile
import shutil

#tidyup
#os.remove(r"C:\mcsig_git\data_load\concelhos.zip")
#shutil.rmtree("C:\mcsig_git\data_load\pt\concelhos")
#os.mkdir("pt\concelhos")
os.remove(r"C:\mcsig_git\data_load\cont-aad-caop2017.zip")
shutil.rmtree(r"C:\mcsig_git\data_load\pt\freguesias")
os.mkdir(r"pt\freguesias")
print('extraneous files removed')

#download Freguesias
print('Beginning freguesias download with wget module')
url = 'https://dados.gov.pt/s/resources/freguesias-de-portugal/20181112-195834/cont-aad-caop2017.zip'
wget.download(url, 'C:\mcsig_git\data_load')
print()
print("downloaded")
with zipfile.ZipFile('C:\mcsig_git\data_load\cont-aad-caop2017.zip', 'r') as zip_ref:
                    zip_ref.extractall(r'C:\mcsig_git\data_load\pt\freguesias')
print('Freguesias unzipped')


extraneous files removed
Beginning freguesias download with wget module
100% [#############################################################]           30M / 30M
downloaded
Freguesias unzipped


In [ ]:
##https://pcjericks.github.io/py-gdalogr-cookbook/vector_layers.html
#https://www.gispo.fi/en/open-software/importing-spatial-data-to-postgis/

import os
from osgeo import ogr, osr

##Define coordinate system as WGS84
#wgs = osr.SpatialReference()
#wgs.ImportFromEPSG(4326)

##Anticipate shape file feature attributes
#driver = ogr.GetDriverByName('ESRI Shapefile')
#shp = driver.CreateDataSource()

##Drop table if already exists
#cur.execute("DROP TABLE IF EXISTS freguesias CASCADE")
#print ("freguesias dropped")

##Create table if exists
#cur.execute("""
#    CREATE TABLE freguesias(
#    dicofre int,
#    freguesia varchar(50),
#    concelho varchar(30),
#    distrito varchar(30),
#    taa varchar(30),
#    area_ea_ha float,
#    area_t_ha float,
#    des_simpli varchar(30),
#    polygon GEOMETRY,
#    
#)
#""")
#conn.commit()
#print("freguesias created")


## Match Spatial
in dev

## Next:

1. Integrate with GIT - DONE
2. Finish adding the full data model - DONE
3. Dimension the data to the model - DONE
4. Load GeoNames to model - DONE
4. Create Lisbon focused gazetteer - DONE
4. Associate spatial 
5. Retreive data in JSON via web service
6. Begin front end